<a href="https://colab.research.google.com/github/mehdi-mustapha/PROJETS-JEDHA/blob/main/features_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load Packages**

In [ ]:
!pip install spacy -q
!python -m spacy download fr_core_news_sm -q
!pip install unidecode
!pip install unidecode
!pip install wordcloud -q
!pip install stylecloud
!pip install matplotlib==3.4
!pip install random2

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
import fr_core_news_sm
import string
import unidecode
from unidecode import unidecode
import re
import pandas as pd
import json
from fnmatch import fnmatch
import random

# Import matplotlib and wordcloud 
import chardet
import spacy
import fr_core_news_sm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import wordcloud
from spacy.lang.fr.stop_words import STOP_WORDS
import numpy as np
from PIL import Image
from collections import Counter


import warnings; warnings.filterwarnings(action='ignore')
import time


import seaborn as sns
%matplotlib inline
large = 18; med = 14; small = 11
params = {'axes.titlesize': large,
          'legend.fontsize': small,
          'figure.figsize': (16, 8),
          'axes.labelsize': small,
          'axes.titlesize': small,
          'xtick.labelsize': small,
          'ytick.labelsize': small,
          'figure.titlesize': med}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

# **Load keyword files**

All keywords have been exported from Google Ads and already cleaned

In [ ]:
#Loading kewywords
kw = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_collection/results/correct_keywords.csv')
kw['kw_len'] = [len(x.split(' ')) for x in kw.keyword]
kw = kw.sort_values(by='kw_len', ascending=False)[['keyword', 'volume']].values
kw[1:5]

# **Load positions and urls from 0 to 100**

In [ ]:
from fnmatch import fnmatch
#list files related to serp data ins "liste"
root = '/content/drive/MyDrive/111_goldfinger/data_collection'
pattern = "*serp*"
i=0
liste = []
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            liste.append(name)
            i+=1
liste.append('last.csv')

In [ ]:
os.chdir('/content/drive/MyDrive/111_goldfinger/data_collection')
df = pd.DataFrame()
for e in liste:
  df = pd.concat([df, pd.read_csv(e)], axis=0)

In [ ]:
df.sample()

In [ ]:
# There are more than 1500 columns such as 'fileFormat',
# 'formattedSearchTime', 'formattedTotalResults', 'formattedUrl',
# 'htmlFormattedUrl', 'htmlSnippet', 'htmlTitle', 'inputEncoding', 'kind',
# 'mime', 'outputEncoding', 'pagemap', 'safe', 'searchTime', 'startIndex',
# 'cse_thumbnail', 'metatags', 'cse_image', 'listitem', 'BreadcrumbList',
# 'article', 'msapplication-tilecolor', 'og:image', 'og:type',
# 'og:image:width', 'fb:app_id', 'theme-color', 'viewport', 'og:title',
# 'og:image:height', 'og:url', 'og:description', 'format-detection',
# 'twitter:app:id:googleplay'... We'll then select the most useful for ou analysis

df = df[['searchTerms', 'rank', 'title','displayLink', 'link']] 

In [ ]:
# Drop records with no availaible url
df.dropna(subset=['link'], inplace=True)
print(df.shape)

# **Load pages from 0 to 100**

In [ ]:
# Loading all scraped pages from position 0 to 100

pages_liste = ['out_of_top.csv', 'lot1.csv', 'lot2.csv', 'lot3.csv', 'lot4.csv', '11_to_20.csv'] #préférer les fichiers json

In [ ]:
os.chdir('/content/drive/MyDrive/111_goldfinger/data_collection/all_pages')
dfr = pd.DataFrame()
for e in pages_liste:
  print(e)
  dfr = pd.concat([dfr, pd.read_csv(e,error_bad_lines=False, skip_blank_lines=True, engine="python")], axis=0)

In [ ]:
dfr.shape

In [ ]:
# Merge position and pages content
data = dfr.merge(df, left_on='raw_url', right_on='link', how='left')  #à reprendre quand integ en json
data.sample()

In [ ]:
data.sample()

In [ ]:
data = data.drop(columns = ['title_y']).rename(columns={'title_x' :'title'})
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

In [ ]:
os.chdir('/content/drive/MyDrive/111_goldfinger/data_processing')
data.to_csv('brut_data.csv')

# **Semantic scores calculation** 

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_processing/brut_data.csv')
data = data[~data['rank'].isna()] #drop lines with no position

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
#content
data['content_len']               = [len(str(x)) for x in data.content]

#title
data['title_len_char']             = [len(str(x)) for x in data.title]

#h1
data['h1_len']                    = [len(str(x).split(' ')) for x in data.h1]

 
#h2, h3
data['nb_h2'] 					          = [len(re.findall('@@',str(x))) for x in data.h2]
data['nb_h3'] 					          = [len(re.findall('@@',str(x))) for x in data.h3]

#links
data['nb_links']                  = [len(re.findall('@@',str(x))) for x in data.nav_links_text]
data['has_canonical']             = [1 if isinstance(x, str) else 0 for x in data.canonical]
data['is_top_ten']                = [1 if x < 11 else 0 for x in data['rank']]


# data.to_csv('+++basic.csv')  

In [ ]:
data = data.drop(columns= ['Unnamed: 0', 'canonical', 'h2', 'h3', 'h4', 'meta_desc',
        'title', 'searchTerms', 'nav_links_text', 'rank'])

In [ ]:
data.to_csv('data_without_score.csv')

adding semantic scores for title & content

In [ ]:
# Check if the kw list is loaded
kw[random.randint(0, len(kw))]

In [ ]:
# Custom function to estimate semantic relevance
# Goal :  evaluate if any of keywords is in document 
# Returns a score based on number of (Google Ads) keywords in text x volume (of search) / text lenght

def word_score(doc, kw):
  score = 0
  tokens = []
  try:
    if isinstance(doc, str):
      doc = doc.lower()
      if '@@' in doc:
        doc = doc.split('@@')
        
      initial_len = len(doc.split(' '))
      
      for e in kw:
        if doc is not None:
          try:
            if len(doc.replace(e[0], '')) < len(doc):
              score += e[1]
              tokens += e[0] + '@@'
              doc = doc.replace(e[0], '')
          except:
            pass

      score = score / initial_len
  except:
    score = 0
    tokens = []

  
  return score

In [ ]:
data['content_score']             = [word_score(x, kw) for x in data.content]
data['title_score']               = [word_score(x, kw) for x in data.title]

In [ ]:
data.sample()

In [ ]:
data.to_csv('/content/drive/MyDrive/111_goldfinger/data_processing/data_w_score.csv')

# **Append page rank data estimates**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_processing/data_w_score.csv')

In [ ]:
pr = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_collection/results/page_rank_done.csv').drop(columns='Unnamed: 0')

In [ ]:
data = data.merge(pr, left_on='displayLink', right_on='dns', how='left')

In [ ]:
data = data.drop(columns=['Unnamed: 0', 'content', 'h1', 'header_links_href', 'header_links_text',
       'raw_url', 'displayLink', 'link', 
        'title',  'dns']).rename(columns={'score' : 'page_rank'}).drop_duplicates()

In [ ]:
data.drop_duplicates().fillna(0).drop(columns=['Unnamed: 0.1']).to_csv('/content/drive/MyDrive/111_goldfinger/data_processing/experimental.csv')

# **Fetch  & append Majestic Rank, Referring Subnets, Referring IPs data**

Selecting most frequent domains (> 93% of dataset)

Builtwith offers a splendid api for fetching tech data points.
We're going to collect some additionnal features (and check if they are relevant for analysis):
```
* technologies'  = len(r.json()['Results'][0]['Result']['Paths'][0]['Technologies'])
* 'mjrank'       = r.json()['Results'][0]['Meta']['MJRank']                           #Majestic Rank between 1 and 1,000,000
* 'a_rank'        = r.json()['Results'][0]['Meta']['ARank']                           #Page Rank between 1 and 100,000,000
* 'ref_sn'        = r.json()['Results'][0]['Attributes']['RefSN']                     #Referring Subnets to Domain from Majestic
* 'ref_ip'        = r.json()['Results'][0]['Attributes']['RefIP']                     #Referring IPs to Domain from Majestic
```



Selecting most frequent domains (> 93% of dataset)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/111_goldfinger/data_processing/data_w_score.csv')

In [ ]:
#Enumeration of unique domains
df = df[['displayLink', 'link']]
df.displayLink.unique()
print(len(df.displayLink.unique()), 'unique domains')

In [ ]:
# What are the dns that come up most often?
count_dns = df.displayLink.value_counts().to_frame()
count_dns['cum_sum'] = count_dns['displayLink'].cumsum()
count_dns['cum_perc'] = 100*count_dns['cum_sum']/count_dns['displayLink'].sum()
count_dns = count_dns[count_dns.displayLink	 > 10]
count_dns

In [ ]:
import os
import requests
from time import sleep
bw_key = open("/content/drive/MyDrive/111_goldfinger/data_collection/bw_key", "r").read()
tech_features = pd.DataFrame()

for elem in count_dns.index:
  e = elem.replace('www.', '')
  url = 'https://api.builtwith.com/v19/api.json?KEY='  + bw_key + '&LOOKUP=' +e

  r = requests.get(url)
  try:
    technologies  = len(r.json()['Results'][0]['Result']['Paths'][0]['Technologies'])
  except:
    technologies =0
  try:
    majestic      = r.json()['Results'][0]['Meta']['Majestic'] 
  except:
    majestic = 0
  try:
    mjrank         = r.json()['Results'][0]['Attributes']['MJRank']
  except:
    mjrank = 0
  try:
    a_rank        = r.json()['Results'][0]['Meta']['ARank']
  except:
    a_rank = 0
  try:                              
    ref_sn        = r.json()['Results'][0]['Attributes']['RefSN']
  except:
    ref_sn = 0
  try:                  
    ref_ip        = r.json()['Results'][0]['Attributes']['RefIP']
  except:
    ref_ip = 0
  try:                 
    spend         = r.json()['Results'][0]['Result']['Spend']
  except:
    spend = 0

  tech = {
  'dns'           : elem,
  'technologies'  : technologies,
  'majestic'      : majestic,
  'mjrank'        : mjrank,                          
  'a_rank'        : a_rank,                          
  'ref_sn'        : ref_sn,                    
  'ref_ip'        : ref_ip,                    
  'spend'         : spend,                    
  }

  tech_features = pd.concat([tech_features,pd.DataFrame.from_dict(data=tech, orient='index').T], axis=0 )
  sleep(0.5)
tech_features.to_csv('/content/drive/MyDrive/111_goldfinger/data_collection/builtwith.csv')
tech_features

In [ ]:
# Merging DataFrames
df_with_tech= df.merge(tech_features, left_on='displayLink', right_on='dns', how='left')

In [ ]:
df_with_tech = df_with_tech.drop(columns= ['Unnamed: 0', 'Unnamed: 0.1', 'content', 'h1', 'header_links_href',
       'header_links_text', 'raw_url', 'displayLink', 'link','title','dns'])

In [ ]:
df_with_tech.sample().T

In [ ]:
df_with_tech.to_csv('tech.csv')

# **Load keyword files**